## Contents
- Column selection
- Row selection with `.loc` and `.iloc`
- Boolean masks and chaining vs. single-expression filters
- Using `query()` for readable filters

In [ ]:
# Setup: imports and display options
import pandas as pd
import numpy as np

pd.options.display.max_columns = 50
pd.options.display.width = 120
pd.options.display.max_rows = 20

print("Setup complete!")
print(f"pandas version: {pd.__version__}")

## 1. Load Raw Datasets

We'll start fresh with the original data:

**Note**: You could load cleaned data from notebook 03 (see commented section at end).

In [ ]:
# Load media contacts data
media_df = pd.read_csv('../data/media_contacts.csv')

print(f"Media data: {media_df.shape}")
print(f"Columns: {media_df.columns.tolist()}")
media_df.head()

In [ ]:
# Load demographic data
demo_df = pd.read_csv('../data/socio_demos.csv')

print(f"Demo data: {demo_df.shape}")
print(f"Columns: {demo_df.columns.tolist()}")
demo_df.head()

## 2. Harmonizing Column Names

**Problem**: Notice the ID columns have different names:
- Media: `PERSON ID` (with space)
- Demo: `Person ID` (with space, different capitalization)

Before merging, we need to standardize these.

In [ ]:
# Check the exact column names
print("Media ID column:", [col for col in media_df.columns if 'ID' in col.upper()])
print("Demo ID column:", [col for col in demo_df.columns if 'ID' in col.upper()])

### Standardize All Column Names

In [ ]:
# Standardize media columns
media_df.columns = media_df.columns.str.strip().str.lower().str.replace(' ', '_')

# Standardize demo columns  
demo_df.columns = demo_df.columns.str.strip().str.lower().str.replace(' ', '_')

print("Standardized columns:")
print(f"Media: {media_df.columns.tolist()}")
print(f"\nDemo: {demo_df.columns.tolist()}")

In [ ]:
# Verify we now have matching ID columns
print("\nID columns after standardization:")
print(f"Media: 'person_id' in columns = {'person_id' in media_df.columns}")
print(f"Demo: 'person_id' in columns = {'person_id' in demo_df.columns}")

## 3. Understanding Merge Types

pandas `merge()` works like SQL joins. Let's explore each type:

### Key Concepts

- **left**: Keep all rows from left DataFrame, match from right
- **right**: Keep all rows from right DataFrame, match from left  
- **inner**: Keep only rows that match in both DataFrames
- **outer**: Keep all rows from both DataFrames

Let's see this with examples!

### Inner Join (Default)

Only keeps rows where `person_id` exists in BOTH datasets:

In [ ]:
# Inner join - only matching rows
merged_inner = pd.merge(
    media_df, 
    demo_df, 
    on='person_id',
    how='inner'
)

print(f"Original sizes:")
print(f"  Media: {len(media_df)} rows")
print(f"  Demo: {len(demo_df)} rows")
print(f"\nAfter inner join: {len(merged_inner)} rows")
print(f"Columns: {len(merged_inner.columns)}")

merged_inner.head()

In [ ]:
# Check if all people matched
print(f"Did all media people match? {len(merged_inner) == len(media_df)}")
print(f"Did all demo people match? {len(merged_inner) == len(demo_df)}")

### Left Join

Keeps ALL rows from the **left** DataFrame (media_df), adds demo data where available:

In [ ]:
# Left join - keep all media rows
merged_left = pd.merge(
    media_df,
    demo_df,
    on='person_id',
    how='left'
)

print(f"After left join: {len(merged_left)} rows")
print(f"Same as media_df? {len(merged_left) == len(media_df)}")

merged_left.head()

In [ ]:
# Check for unmatched rows (NaN in demo columns)
unmatched = merged_left[merged_left['gender'].isna()]
print(f"\nPeople in media but NOT in demo: {len(unmatched)}")
if len(unmatched) > 0:
    print(unmatched.head())

### Right Join

Keeps ALL rows from the **right** DataFrame (demo_df), adds media data where available:

In [ ]:
# Right join - keep all demo rows
merged_right = pd.merge(
    media_df,
    demo_df,
    on='person_id',
    how='right'
)

print(f"After right join: {len(merged_right)} rows")
print(f"Same as demo_df? {len(merged_right) == len(demo_df)}")

merged_right.head()

In [ ]:
# Check for unmatched rows (NaN in media columns)
unmatched_right = merged_right[merged_right['tv_total'].isna()]
print(f"\nPeople in demo but NOT in media: {len(unmatched_right)}")
if len(unmatched_right) > 0:
    print(unmatched_right.head())

### Outer Join (Full Join)

Keeps ALL rows from BOTH DataFrames:

In [ ]:
# Outer join - keep everything
merged_outer = pd.merge(
    media_df,
    demo_df,
    on='person_id',
    how='outer'
)

print(f"After outer join: {len(merged_outer)} rows")
print(f"\nBreakdown:")
print(f"  Original media: {len(media_df)}")
print(f"  Original demo: {len(demo_df)}")
print(f"  After outer: {len(merged_outer)}")

merged_outer.head()

## 4. Detecting Unmatched Rows with Indicator

Use `indicator=True` to see which rows matched and which didn't:

In [ ]:
# Merge with indicator
merged_indicator = pd.merge(
    media_df,
    demo_df,
    on='person_id',
    how='outer',
    indicator=True
)

# Check the indicator values
print("Merge indicator counts:")
print(merged_indicator['_merge'].value_counts())
print("\nIndicator meanings:")
print("  both = matched in both datasets")
print("  left_only = in media but not demo")
print("  right_only = in demo but not media")

In [ ]:
# View rows that didn't match
left_only = merged_indicator[merged_indicator['_merge'] == 'left_only']
print(f"\nRows only in media: {len(left_only)}")
if len(left_only) > 0:
    print(left_only.head())

In [ ]:
# View rows only in demo
right_only = merged_indicator[merged_indicator['_merge'] == 'right_only']
print(f"Rows only in demo: {len(right_only)}")
if len(right_only) > 0:
    print(right_only.head())

## 5. Handling Duplicate Column Names

What if both DataFrames have a column with the same name (other than the merge key)?

Let's demonstrate with an example:

In [ ]:
# Create test dataframes with overlapping column names
df1 = pd.DataFrame({
    'person_id': [1, 2, 3],
    'name': ['Alice', 'Bob', 'Charlie'],
    'score': [85, 90, 88]
})

df2 = pd.DataFrame({
    'person_id': [1, 2, 4],
    'name': ['Alice A.', 'Bob B.', 'David'],
    'grade': ['A', 'A', 'B']
})

print("DataFrame 1:")
print(df1)
print("\nDataFrame 2:")
print(df2)

In [ ]:
# Merge - notice the 'name' column appears in both
merged_dup = pd.merge(df1, df2, on='person_id', how='outer')

print("\nAfter merge (default suffixes):")
print(merged_dup)
print(f"\nColumns: {merged_dup.columns.tolist()}")

### Custom Suffixes

Use `suffixes=` to control how duplicate columns are named:

In [ ]:
# Use custom suffixes
merged_custom = pd.merge(
    df1, 
    df2, 
    on='person_id', 
    how='outer',
    suffixes=('_test1', '_test2')
)

print("With custom suffixes:")
print(merged_custom)
print(f"\nColumns: {merged_custom.columns.tolist()}")

## 6. Building the Final Merged Dataset

For our analysis, we'll create a complete dataset merging media and demographics.

We'll use an **inner join** since we only want people who have both media exposure and demographic data.

In [ ]:
# Create final merged dataset
final_merged = pd.merge(
    media_df,
    demo_df,
    on='person_id',
    how='inner',
    validate='1:1'  # Ensure one-to-one relationship
)

print(f"Final merged dataset: {final_merged.shape}")
print(f"\nColumn summary:")
print(f"  Total columns: {len(final_merged.columns)}")
print(f"  From media: ~{len(media_df.columns)}")
print(f"  From demo: ~{len(demo_df.columns)}")
print(f"  (person_id counted once)")

In [ ]:
# View the merged data
final_merged.head()

In [ ]:
# Check data types
print("Data types in merged dataset:")
print(final_merged.dtypes)

### Validate Merge Quality

In [ ]:
# Check for any missing values introduced by merge
print("Missing values after merge:")
missing = final_merged.isnull().sum()
print(missing[missing > 0])

if missing.sum() == 0:
    print("\n✓ No missing values introduced by merge!")

In [ ]:
# Check for duplicates
duplicates = final_merged.duplicated(subset='person_id').sum()
print(f"\nDuplicate person_ids: {duplicates}")

if duplicates == 0:
    print("✓ No duplicate IDs - clean one-to-one merge!")

## 7. Merge on Multiple Columns

Sometimes you need to merge on more than one column:

In [ ]:
# Example: merge on multiple columns
# Suppose we had region data too

df_region1 = pd.DataFrame({
    'person_id': [1, 1, 2, 2],
    'year': [2024, 2025, 2024, 2025],
    'region': ['North', 'North', 'South', 'South']
})

df_region2 = pd.DataFrame({
    'person_id': [1, 1, 2, 2],
    'year': [2024, 2025, 2024, 2025],
    'sales': [100, 150, 200, 250]
})

# Merge on both person_id AND year
multi_merge = pd.merge(
    df_region1,
    df_region2,
    on=['person_id', 'year'],
    how='inner'
)

print("Merge on multiple columns:")
print(multi_merge)

## 8. Merge with Different Column Names

Use `left_on=` and `right_on=` when merge keys have different names:

In [ ]:
# Example with different column names
customers = pd.DataFrame({
    'customer_id': [101, 102, 103],
    'name': ['Alice', 'Bob', 'Charlie']
})

orders = pd.DataFrame({
    'order_id': [1, 2, 3],
    'cust_id': [101, 102, 104],
    'amount': [50, 75, 100]
})

# Merge when key columns have different names
diff_merge = pd.merge(
    customers,
    orders,
    left_on='customer_id',
    right_on='cust_id',
    how='inner'
)

print("Merge with different key names:")
print(diff_merge)

## 9. Performance Considerations

For large datasets:

In [ ]:
# Check merge performance
import time

start = time.time()
test_merge = pd.merge(media_df, demo_df, on='person_id', how='inner')
elapsed = time.time() - start

print(f"Merge completed in {elapsed:.4f} seconds")
print(f"Rows processed: {len(media_df):,}")
print(f"Merge rate: {len(media_df)/elapsed:,.0f} rows/second")

### Tips for Faster Merges

```python
# 1. Ensure merge columns are the same data type
media_df['person_id'] = media_df['person_id'].astype('int64')
demo_df['person_id'] = demo_df['person_id'].astype('int64')

# 2. Set index on merge column for repeated merges
media_indexed = media_df.set_index('person_id')
demo_indexed = demo_df.set_index('person_id')
merged = media_indexed.join(demo_indexed, how='inner')

# 3. Use categorical dtype for merge keys with few unique values
# (Not applicable here, but useful for region, country, etc.)
```

## Summary

In this notebook, you learned:

✅ Harmonize column names before merging (`.str.lower()`, `.str.replace()`)  
✅ Use `pd.merge()` with different join types (inner, left, right, outer)  
✅ Understand when to use each join type  
✅ Detect unmatched rows with `indicator=True`  
✅ Handle duplicate column names with `suffixes=`  
✅ Merge on single or multiple columns  
✅ Merge with different column names using `left_on=` and `right_on=`  
✅ Validate merge quality (check duplicates, missing values)  
✅ Build analysis-ready merged datasets

### Key Takeaways

1. **Always harmonize column names first**: Prevents merge errors
2. **Choose the right join type**: Think about your analysis needs
3. **Use indicator=True**: Helps detect data quality issues
4. **Validate after merging**: Check for duplicates and unexpected nulls
5. **Document your merge logic**: Comment why you chose left/inner/outer

### Next Steps

In the next notebook (**05_groupby_and_aggregation.ipynb**), we'll:
- Use this merged dataset for analysis
- Group data by demographics
- Calculate aggregated statistics
- Compute purchase rates by segment
- Create summary tables

## 🎯 Practice Exercises

Try these on your own:

1. Create a left join keeping all media rows - how many have missing demo data?
2. Merge with `indicator=True` and count rows in each category (both, left_only, right_only)
3. Create an outer join and fill missing media columns with 0
4. Merge only people aged 25-50 from demo with all media data
5. Create a merge that keeps only females with high TV exposure (>30)
6. Check if there are any `person_id` values that appear multiple times in either dataset
7. Create a custom suffixes merge and rename the suffix columns afterward
8. Merge and then filter to only purchasers (`purchase == 1`)

### Bonus Challenges

9. Create age bands in demo_df first, then merge and analyze purchase rates by age band
10. Merge three datasets (media, demo, and a hypothetical third dataset)
11. Perform a merge, then groupby a demographic variable to find average TV exposure
12. Create a validation function that checks merge quality (duplicates, nulls, row counts)

## Loading/Saving Data Between Notebooks

### Load Cleaned Data from Previous Notebook

If you cleaned data in notebook 03:

```python
# Uncomment to load cleaned data
# media_df = pd.read_csv('../outputs/media_cleaned.csv')
# demo_df = pd.read_csv('../outputs/demo_cleaned.csv')
#
# # Ensure person_id exists (adjust based on your cleaning)
# if 'person_id' not in media_df.columns:
#     media_df.columns = media_df.columns.str.lower().str.replace(' ', '_')
```

### Save Merged Dataset for Next Notebooks

Save your merged data for use in analysis notebooks:

```python
# Uncomment to save merged data
# import os
# os.makedirs('../outputs', exist_ok=True)
#
# # Save as CSV
# final_merged.to_csv('../outputs/merged_data.csv', index=False)
#
# # Or save as Parquet (faster, smaller, preserves dtypes)
# final_merged.to_parquet('../outputs/merged_data.parquet')
#
# print(f"Saved merged dataset: {final_merged.shape}")
```

**Tip**: Parquet format is recommended for intermediate files - it's faster to read/write and preserves data types!